In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Some EDA and cleaning on the way

In [2]:
df = pd.read_csv('breast_cancer_staging.csv')
df.head()

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (6,11,13,18,25,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,file_name,case_id,submitter_id,project_id,gender,year_of_birth,race,days_to_birth,ethnicity,...,ENSGR0000275287.3,ENSGR0000276543.3,ENSGR0000277120.3,ENSGR0000280767.1,ENSGR0000281849.1,__alignment_not_unique,__ambiguous,__no_feature,__not_aligned,__too_low_aQual
0,0,2b5c518f-8327-478a-a282-01bfe59aca4c.htseq.cou...,4d0fec97-e024-4608-a0cc-426a3decc7b1,TCGA-AR-A252,TCGA-BRCA,female,1957,white,-18611,not hispanic or latino,...,0.0,0.0,0.0,0.0,0.0,33323894.0,3232629.0,5808920.0,0.0,0.0
1,1,64262505-9f17-4989-94c9-fc6db56ca676.htseq.cou...,bb8d42d3-ad65-4d88-ae1d-f9aadfc7962d,TCGA-AO-A1KS,TCGA-BRCA,female,1941,white,-25230,not hispanic or latino,...,0.0,0.0,0.0,0.0,0.0,19183901.0,2673271.0,6146344.0,0.0,0.0
2,2,f5d3e683-9177-45fc-93e3-357bf7366ac4.htseq.cou...,c694615c-b1c6-499c-8058-995633ebf948,TCGA-E9-A1RF,TCGA-BRCA,female,1943,white,-25119,not hispanic or latino,...,0.0,0.0,0.0,0.0,0.0,23874394.0,2400693.0,5278313.0,0.0,0.0
3,3,94bedc10-62c2-4bd2-bad9-0ec08c9bf5e9.htseq.cou...,c694615c-b1c6-499c-8058-995633ebf948,TCGA-E9-A1RF,TCGA-BRCA,female,1943,white,-25119,not hispanic or latino,...,0.0,0.0,0.0,0.0,0.0,21737695.0,3123055.0,5182286.0,0.0,0.0
4,4,6999d309-8502-49ee-8d80-9d0bde00081f.htseq.cou...,1285eb55-415c-494a-aa58-936f0427cdd0,TCGA-V7-A7HQ,TCGA-BRCA,female,1934,black or african american,-27684,not hispanic or latino,...,0.0,0.0,0.0,0.0,0.0,20631886.0,2996831.0,2104393.0,0.0,0.0


# Gens expressions

In [20]:
def check_significant_features(feature):
    print(feature.value_counts())
    
gens = df.drop(columns=['Unnamed: 0', 'file_name', 'case_id', 'submitter_id', 'project_id',
       'gender', 'year_of_birth', 'race', 'days_to_birth', 'ethnicity', 'vital_status', 'days_to_death', 'age_at_index', 'year_of_death',
       'year_of_diagnosis', 'classification_of_tumor',
       'last_known_disease_status', 'primary_diagnosis', 'tumor_stage',
       'age_at_diagnosis', 'morphology', 'days_to_last_known_disease_status', 'prior_treatment',
       'days_to_recurrence', 'tumor_grade', 'icd_10_code', 'days_to_diagnosis',
       'tissue_or_organ_of_origin', 'progression_or_recurrence',
       'prior_malignancy', 'synchronous_malignancy', 'site_of_resection_or_biopsy',
       'days_to_last_follow_up', 'days_to_treatment_start', 'treatment_effect',
       'initial_disease_status', 'treatment_type', 'therapeutic_agents',
       'regimen_or_line_of_therapy', 'treatment_intent_type', 'treatment_anatomic_site', 'treatment_outcome', 'days_to_treatment_end',
       'treatment_or_therapy', '__alignment_not_unique',
       '__ambiguous', '__no_feature', '__not_aligned', '__too_low_aQual'])

# gens.apply(check_significant_features, axis=0)

gens_varieties = gens.nunique()

In [37]:
percentage_of_non_varying = np.round(float(len(gens_varieties[gens_varieties == 1]))*100/len(gens_varieties), 2)
print(f"""The total number of gens we have is {len(gens_varieties)}
But {percentage_of_non_varying} % have the same value for all the dataset""")

The total number of gens we have is 60483
But 3.67 % have the same value for all the dataset


In [41]:
insignificant_gens = np.array(gens_varieties[gens_varieties == 1].index)
insignificant_gens

array(['ENSG00000100146.15', 'ENSG00000100219.15', 'ENSG00000100249.4',
       ..., 'ENSGR0000277120.3', 'ENSGR0000280767.1', 'ENSGR0000281849.1'],
      dtype=object)

### Dropping unsignificant gens from df

In [42]:
df = df.drop(columns=insignificant_gens)

In [43]:
df.shape

(1164, 58312)

In [49]:
df['ENSG00000281910.1'].value_counts()

0.0     979
1.0     128
2.0      32
4.0       7
3.0       4
6.0       3
19.0      2
24.0      2
10.0      2
11.0      2
17.0      1
5.0       1
13.0      1
Name: ENSG00000281910.1, dtype: int64

# Clinical info


In [50]:
df.columns

Index(['Unnamed: 0', 'file_name', 'case_id', 'submitter_id', 'project_id',
       'gender', 'year_of_birth', 'race', 'days_to_birth', 'ethnicity',
       ...
       'ENSG00000281910.1', 'ENSG00000281912.1', 'ENSG00000281918.1',
       'ENSG00000281920.1', 'ENSG00000281921.1', '__alignment_not_unique',
       '__ambiguous', '__no_feature', '__not_aligned', '__too_low_aQual'],
      dtype='object', length=58312)

### Year of birth

In [62]:
df['year_of_birth'].astype(str).str.len().value_counts()

4    1162
2       2
Name: year_of_birth, dtype: int64

So there is 2 rows where the years have 2 digits

In [64]:
df[df['year_of_birth'].astype(str).str.len() == 2]['year_of_birth']

762    --
866    --
Name: year_of_birth, dtype: object

We should replace this for nan

In [66]:
indexes_to_replace = df[df['year_of_birth'].astype(str).str.len() == 2]['year_of_birth'].index
df.loc[indexes_to_replace, 'year_of_birth'] = np.nan

In [75]:
df['year_of_birth'].value_counts().index

Index([  1953,   1946,   1948,   1964,   1949,   1947,   1951,   1945,   1960,
         1962,   1940,   1954,   1952,   1957,   1958,   1943,   1944,   1967,
         1963,   1950,   1941,   1939,   1956,   1955,   1966,   1961,   1936,
         1959,   1965,   1931,   1942,   1928,   1938,   1933,   1969,   1934,
         1968,   1929,   1930,   1972,   1924,   1937,   1932,   1971,   1970,
         1935,   1925,   1922,   1926,   1921,   1927,   1919,   1920,   1974,
         1977,   1973,   1976,   1923,   1917,   1918,   1978,   1979, '1916',
       '1920',   1983,   1984, '1938',   1916, '1954',   1981,   1913, '1950',
         1910, '1952',   1909, '1942', '1965', '1937',   1914, '1946', '1941',
       '1940',   1980, '1951', '1948', '1944',   1975, '1930', '1969', '1958',
       '1939', '1924',   1902, '1947', '1966', '1928', '1960'],
      dtype='object')

We see that they are still kind of messy, some of them have '' others not

In [81]:
df['year_of_birth'] = df['year_of_birth'].str.strip("'")
df['year_of_birth'].value_counts().index

Index(['1954', '1938', '1916', '1952', '1920', '1950', '1951', '1941', '1947',
       '1966', '1960', '1969', '1928', '1940', '1946', '1930', '1958', '1924',
       '1939', '1937', '1944', '1965', '1942', '1948'],
      dtype='object')

### Race

In [84]:
df['race'].value_counts()

white                               832
black or african american           180
not reported                         94
asian                                57
american indian or alaska native      1
Name: race, dtype: int64

What can we do with this only `american indian or alaska native` row? Drop it?

In [85]:
df['ethnicity'].value_counts()

not hispanic or latino    918
not reported              195
hispanic or latino         36
Alive                      15
Name: ethnicity, dtype: int64

# Persist it after changing it

In [82]:
df.to_csv('cleaned_dataset.csv')

KeyboardInterrupt: 